In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns

In [ ]:
tond = pd.read_csv("type_before_pre_ton.csv")

In [ ]:
iot20 = pd.read_csv("iot2020.csv")

In [ ]:
#이종의 데이터셋을 5:5비율로 학습하기 위해 Ton-IoT 데이터셋의 개수만큼 랜덤 추출
iot20 = iot20.sample(n=461043, replace=False, axis=0)

In [ ]:
#공격 종류 확인
tond['type'].value_counts()

In [ ]:
#공격 종류 확인
iot20['Cat'].value_counts()

In [ ]:
#type과 Cat이 둘 다 공격명을 나타내므로 Column명 통일
#iot20과 ton-iot 공격명 통일
tond = tond.rename(columns={'type':'Cat'})
iot20 = iot20.replace('Normal', 'normal' )
iot20 = iot20.replace('DoS', 'dos' )
iot20 = iot20.replace('MITM ARP Spoofing', 'mitm' )
iot20 = iot20.replace('Scan', 'scanning' )

In [ ]:
#feature명 통일
tond= tond.rename(columns={'duration':'Flow_Duration','src_pkts':'Tot_Fwd_Pkts', 'dst_pkts':'Tot_Bwd_Pkts', 'proto':'Protocol', 'src_bytes':'Fwd_Pkt_Len_Mean' , 'dst_bytes':'Bwd_Pkt_Len_Mean', 'src_ip_bytes':'Fwd_Pkts/s', 'dst_ip_bytes':'Bwd_Pkts/s', 'type':'Cat'})

In [ ]:
#feature명 통일
iot20= iot20.rename(columns={'duration':'Flow Duration','orig_pkts':'Total Fwd Packets', 'resp_pkts':'Total Backward Packets', 'proto':'Protocol', 'orig_bytes':'Fwd Packet Length Mean' , 'resp_bytes': 'Bwd Packet Length Mean' , 'orig_ip_bytes': 'Fwd Packets/s' ,'resp_ip_bytes': 'Bwd Packets/s' ,'label': 'Label' })

In [ ]:
#String값을 가진 feature drop
iot20 = iot20.drop(columns=[ 'Flow_Byts/s', 'Flow_Pkts/s',  'Label', 'Sub_Cat' ])

In [ ]:
print(tond.columns.tolist())

In [ ]:
print(iot20.columns.tolist())

In [ ]:
#공통 feature 9개를 제외한 나머지 drop
iot20 = iot20.drop(columns=['Flow_ID', 'Src_IP', 'Src_Port', 'Dst_IP', 'Dst_Port', 'Timestamp', 'TotLen_Fwd_Pkts', 'TotLen_Bwd_Pkts', 'Fwd_Pkt_Len_Max','Fwd_Pkt_Len_Min', 'Fwd_Pkt_Len_Std', 'Bwd_Pkt_Len_Max', 'Bwd_Pkt_Len_Min', 'Bwd_Pkt_Len_Std', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Tot', 'Fwd_IAT_Mean', 'Fwd_IAT_Std', 'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Tot', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Len', 'Bwd_Header_Len', 'Pkt_Len_Min', 'Pkt_Len_Max', 'Pkt_Len_Mean', 'Pkt_Len_Std', 'Pkt_Len_Var', 'FIN_Flag_Cnt', 'SYN_Flag_Cnt', 'RST_Flag_Cnt', 'PSH_Flag_Cnt', 'ACK_Flag_Cnt', 'URG_Flag_Cnt', 'CWE_Flag_Count', 'ECE_Flag_Cnt', 'Down/Up_Ratio', 'Pkt_Size_Avg', 'Fwd_Seg_Size_Avg', 'Bwd_Seg_Size_Avg', 'Fwd_Pkts/b_Avg', 'Fwd_Blk_Rate_Avg', 'Bwd_Byts/b_Avg', 'Bwd_Pkts/b_Avg', 'Bwd_Blk_Rate_Avg', 'Subflow_Fwd_Pkts', 'Subflow_Fwd_Byts', 'Subflow_Bwd_Pkts', 'Subflow_Bwd_Byts', 'Init_Fwd_Win_Byts', 'Init_Bwd_Win_Byts', 'Fwd_Act_Data_Pkts', 'Fwd_Seg_Size_Min', 'Active_Mean', 'Active_Std', 'Active_Max', 'Active_Min', 'Idle_Mean', 'Idle_Std', 'Idle_Max', 'Idle_Min'])

In [ ]:
##공통 feature 9개를 제외한 나머지 drop
tond = tond.drop(columns=['Unnamed: 0', 'ts', 'service', 'conn_state', 'missed_bytes', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD', 'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed', 'ssl_established', 'ssl_subject', 'ssl_issuer', 'http_trans_depth', 'http_method', 'http_version', 'http_request_body_len', 'http_response_body_len', 'http_status_code', 'http_orig_mime_types', 'http_resp_mime_types', 'weird_name', 'weird_addl', 'weird_notice' , 'label', 'src_port', 'dst_port'  ])

In [ ]:
frames = [iot20, tond]
df_h = pd.concat(frames)

In [ ]:
#결측값 존재하는지 확인
tond.isna().sum()

In [ ]:
#결측값 채움
tond.fillna(-1,inplace=True)
iot20.fillna(-1,inplace=True)
df_h.fillna(-1,inplace=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

In [ ]:
#이종의 데이터셋으로 학습
X = df_h[['Protocol', 'Flow_Duration', 'Fwd_Pkt_Len_Mean', 'Bwd_Pkt_Len_Mean', 'Tot_Fwd_Pkts', 'Fwd_Pkts/s', 'Tot_Bwd_Pkts', 'Bwd_Pkts/s']]
Y = df_h['Cat']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [ ]:
#IoTID20으로 테스트하는 경우
Z = iot20[['Protocol', 'Flow_Duration', 'Fwd_Pkt_Len_Mean', 'Bwd_Pkt_Len_Mean', 'Tot_Fwd_Pkts', 'Fwd_Pkts/s', 'Tot_Bwd_Pkts', 'Bwd_Pkts/s']]
W = iot20['Cat']
Z_train, Z_test, W_train, W_test = train_test_split(Z, W, random_state=10, test_size=0.4)

In [ ]:
X_test = Z_test
Y_test = W_test

In [ ]:
start = time.time()
print('program start...')
print()

RF.fit(X_train, Y_train)  # fit() : 훈련 수행
print()
print('prediction:')
y_pred = RF.predict(X_test)  # predict() : 모델 훈련 후 예측
print(y_pred)
print()

print('Score:')
score = RF.score(X_test,Y_test)  # score() : 평가 점수 계산
print(score)

end = time.time()  # 현재 시각 구하기
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))